In [9]:
# Want to predict the firing rates across sessions given the alignment of tuning curves

import os
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
grandparent_directory = os.path.abspath(os.path.join(parent_directory, os.pardir))
os.sys.path.append(grandparent_directory)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import plotly.graph_objects as go

from tools.data_formatting import get_smoothed_moving_all_data
from tools.data_manager import get_all_experiments_runs
from global_vars import order_experiments
from tools.alignment import procrustes, canoncorr

# Load the data

In [ ]:
### Load all sessions ###

animal = 'm120'
fov = 'fov1'
sessions = get_all_experiments_runs(animal, fov)
# Create a mapping of sessions and their chronological order
order_map = {value: index for index, value in enumerate(order_experiments)}
sessions = sorted(sessions, key=lambda x: order_map[x])

all_data = {}
for (experiment, run) in sessions:
    firing_rates, time_unwrapped, phi, cells, tuning_curves, phi_bins = get_smoothed_moving_all_data(animal, fov, experiment, run)
    all_data[(experiment, run)] = {}
    all_data[(experiment, run)]['firing_rates'] = firing_rates
    all_data[(experiment, run)]['time'] = time_unwrapped
    all_data[(experiment, run)]['phi'] = phi
    all_data[(experiment, run)]['cells'] = cells
    all_data[(experiment, run)]['tuning_curves'] = tuning_curves
    all_data[(experiment, run)]['phi_bins'] = phi_bins

In [20]:
# Try among two sessions for now
ref_experiment, ref_run = sessions[0]
experiment, run = sessions[1]
print(f'Using {ref_experiment}-{ref_run} as reference and {experiment}-{run} as target')
ref_fr = all_data[(ref_experiment, ref_run)]['firing_rates']
fr = all_data[(experiment, run)]['firing_rates']
print(f'Firing rates shape: {ref_fr.shape}, {fr.shape}')

# Get the tuning curves embeddings and align them
pca = PCA(n_components=20)
ref_pca_tc = pca.fit_transform(all_data[(ref_experiment, ref_run)]['tuning_curves'])
pca_tc = pca.fit_transform(all_data[(experiment, run)]['tuning_curves'])
pca_components = pca.components_
print(f'PCA manifolds shape: {ref_pca_tc.shape}, {pca_tc.shape}')

# CCA alignment
A, B, r, U, V = canoncorr(ref_pca_tc, pca_tc, fullReturn=True)



Using fam1fam2-fam1 as reference and fam1fam2-fam2 as target
Firing rates shape: (6087, 156), (5391, 156)
PCA manifolds shape: (360, 20), (360, 20)


In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=(ref_pca_tc@A)[:, 0], y=(ref_pca_tc@A)[:, 1], z=(ref_pca_tc@A)[:, 2], mode='markers', marker=dict(size=3, color=phi_bins, colorscale='hsv')))
fig.add_trace(go.Scatter3d(x=(pca_tc@B)[:, 0], y=(pca_tc@B)[:, 1], z=(pca_tc@B)[:, 2], mode='markers', marker=dict(size=3, color=phi_bins, colorscale='hsv')))
fig.show()